In [161]:
# !pip install category_encoders

In [162]:
import pandas as pd
import seaborn as sns
import warnings
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from scipy.stats import norm
from scipy import stats
import statistics as st
from matplotlib.ticker import ScalarFormatter
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import MinMaxScaler
from category_encoders import TargetEncoder

## Data Processing

In [163]:
df = pd.read_csv('SBA_loans_test_1')

In [164]:
df = df.drop('Unnamed: 0', axis = 1)

In [165]:
def data_cleanup(df):
    for col in ['DisbursementGross', 'BalanceGross', 'GrAppv', 'SBA_Appv']:
        df[col] = df[col].apply(lambda x: x.strip('$'))
        df[col] = df[col].apply(lambda x: x.replace(',', ''))
        df[col] = df[col].astype(float)
    
    df['RevLineCr'] = df['RevLineCr'].replace('0', 'N') 
    
    return df

In [166]:
df = data_cleanup(df)

In [167]:
df.head(20)

,index,City,State,Zip,Bank,BankState,NAICS,Term,NoEmp,NewExist,...,RetainedJob,FranchiseCode,UrbanRural,RevLineCr,LowDoc,DisbursementGross,BalanceGross,GrAppv,SBA_Appv,MIS_Status
0,655010,CHANHASSEN,MN,55317,TWIN CITIES-METRO CERT. DEVEL,MN,0,240,20,1.0,...,2,1,1,N,N,518000.0,0.0,518000.0,518000.0,P I F
1,468337,PLANO,TX,75074,COMERICA BANK,TX,447110,84,6,1.0,...,6,1,1,N,N,40000.0,0.0,40000.0,30000.0,P I F
2,389896,HAMMOND,IN,46324,CENTIER BANK,IN,0,144,4,1.0,...,0,1,0,N,N,140500.0,0.0,140500.0,112400.0,P I F
3,600018,MOUNT PLEASANT,SC,29464,BBCN BANK,CA,561790,20,2,2.0,...,2,1,1,N,N,10000.0,0.0,10000.0,8500.0,CHGOFF
4,302415,BOLINGBROOK,IL,60490,HANMI BANK,CA,812320,71,6,1.0,...,4,1,1,N,N,220000.0,0.0,220000.0,165000.0,P I F
5,449356,CARMEL,IN,46032,JPMORGAN CHASE BANK NATL ASSOC,IL,0,84,37,1.0,...,0,77875,0,N,N,100000.0,0.0,100000.0,85000.0,P I F
6,295105,DOVER,DE,19901,BBCN BANK,CA,453220,84,1,1.0,...,1,1,1,N,N,10000.0,0.0,10000.0,8500.0,P I F
7,787443,BOZEMAN,MT,59715,FIRST SECURITY BANK,MT,323115,84,12,1.0,...,0,75000,2,N,N,390000.0,0.0,390000.0,292500.0,P I F
8,136897,HOBBS,NM,88240,CAPITAL CERT. DEVEL CORP,TX,721110,240,1,1.0,...,0,20718,2,N,N,1492000.0,0.0,1492000.0,1492000.0,P I F
9,464835,ALBANY,NY,12207,EMPIRE ST. CERT. DEVEL CORP,NY,0,240,11,1.0,...,0,0,0,N,N,158000.0,0.0,158000.0,158000.0,P I F


In [168]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 647397 entries, 0 to 647396
Data columns (total 21 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   index              647397 non-null  int64  
 1   City               647377 non-null  object 
 2   State              647390 non-null  object 
 3   Zip                647397 non-null  int64  
 4   Bank               646296 non-null  object 
 5   BankState          646290 non-null  object 
 6   NAICS              647397 non-null  int64  
 7   Term               647397 non-null  int64  
 8   NoEmp              647397 non-null  int64  
 9   NewExist           647291 non-null  float64
 10  CreateJob          647397 non-null  int64  
 11  RetainedJob        647397 non-null  int64  
 12  FranchiseCode      647397 non-null  int64  
 13  UrbanRural         647397 non-null  int64  
 14  RevLineCr          644152 non-null  object 
 15  LowDoc             645554 non-null  object 
 16  Di

In [169]:
# Make a copy of original dataset 
copy_df = df.copy()

In [170]:
def imputer(df):
# check columns that contain null values 
    na_num_columns = []
    na_cat_columns = []

    for col in df.columns:
        if df[col].dtype == float:
            if df[col].isna().any() == True:
                na_num_columns.append(col)
        if df[col].dtype == int:
            if df[col].isna().any() == True:
                na_num_columns.append(col)
        if df[col].dtype == object:
            if df[col].isna().any() == True:
                na_cat_columns.append(col)


    # Use the SimpleImputer to handle the null values
    num_imputer = SimpleImputer(strategy = 'most_frequent')
    cat_imputer = SimpleImputer(strategy='most_frequent')

    df[na_num_columns] = num_imputer.fit_transform(df[na_num_columns])
    df[na_cat_columns] = cat_imputer.fit_transform(df[na_cat_columns])

    # Encoding MIS_Status
    df['MIS_Status'] = df['MIS_Status'].replace('CHGOFF', 0)
    df['MIS_Status'] = df['MIS_Status'].replace('P I F', 1)
    df['MIS_Status'] = df["MIS_Status"].astype('int')
    
    return df

In [171]:
copy_df = imputer(copy_df)

In [172]:
copy_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 647397 entries, 0 to 647396
Data columns (total 21 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   index              647397 non-null  int64  
 1   City               647397 non-null  object 
 2   State              647397 non-null  object 
 3   Zip                647397 non-null  int64  
 4   Bank               647397 non-null  object 
 5   BankState          647397 non-null  object 
 6   NAICS              647397 non-null  int64  
 7   Term               647397 non-null  int64  
 8   NoEmp              647397 non-null  int64  
 9   NewExist           647397 non-null  float64
 10  CreateJob          647397 non-null  int64  
 11  RetainedJob        647397 non-null  int64  
 12  FranchiseCode      647397 non-null  int64  
 13  UrbanRural         647397 non-null  int64  
 14  RevLineCr          647397 non-null  object 
 15  LowDoc             647397 non-null  object 
 16  Di

## Split Data into train and test

In [173]:
# Split the data into test and train splits
# Make sure the target split is propertionate for the model training and testing
target_col = copy_df['MIS_Status']

# Setting the stratify to the target_col
train_df, test_df = train_test_split(copy_df, 
                                     test_size=0.2,
                                     random_state=100,
                                     stratify=target_col)

In [174]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 517917 entries, 356940 to 217057
Data columns (total 21 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   index              517917 non-null  int64  
 1   City               517917 non-null  object 
 2   State              517917 non-null  object 
 3   Zip                517917 non-null  int64  
 4   Bank               517917 non-null  object 
 5   BankState          517917 non-null  object 
 6   NAICS              517917 non-null  int64  
 7   Term               517917 non-null  int64  
 8   NoEmp              517917 non-null  int64  
 9   NewExist           517917 non-null  float64
 10  CreateJob          517917 non-null  int64  
 11  RetainedJob        517917 non-null  int64  
 12  FranchiseCode      517917 non-null  int64  
 13  UrbanRural         517917 non-null  int64  
 14  RevLineCr          517917 non-null  object 
 15  LowDoc             517917 non-null  object 
 1

In [194]:
test_df.shape

(129480, 21)

## Encoding

In [176]:
# train_df_copy = train_df.copy()

In [177]:
def encode_below_10(df):
    ohe_cat_cols = []  # categorical columns having less than 10 unique values
    ohe_processed_cols = []

    for col in df.columns:
        if df[col].dtype == object:
            if df[col].nunique() <= 10:
                ohe_cat_cols.append(col)

    ohe_enc = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

    # Fit OHE
    ohe_enc_encoder = ohe_enc.fit(df[ohe_cat_cols])

    # Transform using OHE
    ohe_data_cols = ohe_enc_encoder.transform(df[ohe_cat_cols])

    # Get feature names from the encoder
    feature_names = ohe_enc.get_feature_names_out(ohe_cat_cols)

    # Create a DataFrame from the one-hot-encoded data
    ohe_df = pd.DataFrame(ohe_data_cols, columns=feature_names)

    # Concatenate the original DataFrame with the one-hot-encoded DataFrame
    df = pd.concat([df, ohe_df], axis=1)
    df = df.fillna(value=0)
    df = df.replace(np.inf, 0)
    
    return df


In [178]:
train_df = encode_below_10(train_df)

In [179]:
train_df.head()

,index,City,State,Zip,Bank,BankState,NAICS,Term,NoEmp,NewExist,...,GrAppv,SBA_Appv,MIS_Status,LowDoc_0,LowDoc_A,LowDoc_C,LowDoc_N,LowDoc_R,LowDoc_S,LowDoc_Y
356940,265151.0,BATON ROUGE,LA,70802.0,PLAQUEMINE BK & TR CO,LA,0.0,84.0,12.0,1.0,...,100000.0,90000.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
557194,581236.0,DALLAS,TX,75214.0,"BUSINESS LOAN CENTER, LLC",SC,0.0,240.0,4.0,1.0,...,500000.0,375000.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
217042,420853.0,ROCHESTER,NY,14617.0,FIRST NIAGARA BANK NATL ASSOC,NY,811310.0,60.0,1.0,1.0,...,15000.0,7500.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
238598,770643.0,ROCHESTER,NY,14625.0,CITIZENS BANK NATL ASSOC,RI,423390.0,60.0,1.0,1.0,...,40000.0,20000.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
563153,268102.0,ATTICA,NY,14011.0,FIVE STAR BANK,NY,444130.0,189.0,13.0,2.0,...,505000.0,422938.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [180]:
def encode_above_10(df):
    # For unique is greater than 10
    te  = TargetEncoder()
    tar_cat_cols = []
    for col in df.columns:
        if df[col].dtype == object:
            if df[col].nunique() > 10:
                tar_cat_cols.append(col)
    # Fit Target Encoder
    tar_enc_encoder = te.fit(df[tar_cat_cols], df['MIS_Status']) #fit needs 2 parameters
    # Transform using Target Encoder
    tar_encoded_cols =  te.transform(df[tar_cat_cols])
    # Append Target Encoded columns to train
    for col in tar_cat_cols:
        df[col+"_trg"] = tar_encoded_cols[col]
    # Maintain a list of columns Processed with Target Encoding
    te_processed_cols = ['City_trg', 'State_trg', 'Bank_trg', 'BankState_trg', 'RevLineCr_trg', 'LoanInd_trg']
    
    return df

In [181]:
train_df = encode_above_10(train_df)

In [182]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 621534 entries, 356940 to 517916
Data columns (total 33 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   index              621534 non-null  float64
 1   City               621534 non-null  object 
 2   State              621534 non-null  object 
 3   Zip                621534 non-null  float64
 4   Bank               621534 non-null  object 
 5   BankState          621534 non-null  object 
 6   NAICS              621534 non-null  float64
 7   Term               621534 non-null  float64
 8   NoEmp              621534 non-null  float64
 9   NewExist           621534 non-null  float64
 10  CreateJob          621534 non-null  float64
 11  RetainedJob        621534 non-null  float64
 12  FranchiseCode      621534 non-null  float64
 13  UrbanRural         621534 non-null  float64
 14  RevLineCr          621534 non-null  object 
 15  LowDoc             621534 non-null  object 
 1

## Scaling

In [183]:
# get the columns to scale
cols_to_scale= []

for col in df.columns:
    if df[col].dtype == 'int64':
        cols_to_scale.append(col)

In [184]:
# Scaling continous numerical variables excluding Index and Zip columns
scaler = MinMaxScaler()
cols_to_scale = cols_to_scale[2:]
#taking care of discrepancies which might have appeared after column transformations
train_df = train_df.fillna(value=0)
train_df = train_df.replace(np.inf, 0)

# Fit the scaler to the selected columns
scaler = scaler.fit(train_df[cols_to_scale])
scaled_cols = scaler.transform(train_df[cols_to_scale])
scaled_cols = pd.DataFrame(scaled_cols, columns=cols_to_scale)
train_df = train_df.reset_index(drop=True)
# Append scaled columns to the train frame
for col in scaled_cols:
    train_df[col+'_sc'] = scaled_cols[col]
    train_df.drop(col, axis=1)

In [185]:
train_df.head()

,index,City,State,Zip,Bank,BankState,NAICS,Term,NoEmp,NewExist,...,Bank_trg,BankState_trg,RevLineCr_trg,NAICS_sc,Term_sc,NoEmp_sc,CreateJob_sc,RetainedJob_sc,FranchiseCode_sc,UrbanRural_sc
0,265151.0,BATON ROUGE,LA,70802.0,PLAQUEMINE BK & TR CO,LA,0.0,84.0,12.0,1.0,...,0.762492,0.894986,0.85311,0.000000,0.147627,0.0012,0.0,0.000000,0.000011,0.0
1,581236.0,DALLAS,TX,75214.0,"BUSINESS LOAN CENTER, LLC",SC,0.0,240.0,4.0,1.0,...,0.689877,0.842927,0.85311,0.000000,0.421793,0.0004,0.0,0.000000,0.000011,0.0
2,420853.0,ROCHESTER,NY,14617.0,FIRST NIAGARA BANK NATL ASSOC,NY,811310.0,60.0,1.0,1.0,...,0.932077,0.831558,0.85311,0.874143,0.105448,0.0001,0.0,0.000105,0.000011,0.5
3,770643.0,ROCHESTER,NY,14625.0,CITIZENS BANK NATL ASSOC,RI,423390.0,60.0,1.0,1.0,...,0.784809,0.801270,0.85311,0.456180,0.105448,0.0001,0.0,0.000105,0.000011,0.5
4,268102.0,ATTICA,NY,14011.0,FIVE STAR BANK,NY,444130.0,189.0,13.0,2.0,...,0.895778,0.831558,0.85311,0.478526,0.332162,0.0013,0.0,0.000000,0.000011,0.0


In [186]:
train_df['index'] = train_df['index'].astype('int')
train_df['Zip'] = train_df['Zip'].astype('int')

In [187]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 621534 entries, 0 to 621533
Data columns (total 40 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   index              621534 non-null  int64  
 1   City               621534 non-null  object 
 2   State              621534 non-null  object 
 3   Zip                621534 non-null  int64  
 4   Bank               621534 non-null  object 
 5   BankState          621534 non-null  object 
 6   NAICS              621534 non-null  float64
 7   Term               621534 non-null  float64
 8   NoEmp              621534 non-null  float64
 9   NewExist           621534 non-null  float64
 10  CreateJob          621534 non-null  float64
 11  RetainedJob        621534 non-null  float64
 12  FranchiseCode      621534 non-null  float64
 13  UrbanRural         621534 non-null  float64
 14  RevLineCr          621534 non-null  object 
 15  LowDoc             621534 non-null  object 
 16  Di

## Feature Engineering

In [188]:
def df_feature_engineering(df):
    df['NoEmpByState'] = df.groupby(['BankState', 'Bank'])['NoEmp'].transform('mean')
    df['ApprvLoanDiff'] = abs(df['GrAppv'] - df['SBA_Appv'])
    df['LoanGrossPercentage'] = (df['SBA_Appv']/df['DisbursementGross'])*100
    df['LogGrAppv'] = np.log10(df['GrAppv'])                     
    df['LogSBAAppv'] = np.log10(df['SBA_Appv'])
    df['GrossLogApprvRatio'] = (df['LogSBAAppv']/df['LogGrAppv'])
    df['MISMeanByCity'] = df['MIS_Status'].groupby(df['City']).transform('mean')
    df['MISMeanByState'] = df['MIS_Status'].groupby(df['State']).transform('mean')
    df['LoanApprvByState'] = df['ApprvLoanDiff'].groupby(df['BankState']).transform('mean')
    
    return df

In [189]:
# Suppress runtime warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

train_df = df_feature_engineering(train_df)

In [190]:
train_df.head()

,index,City,State,Zip,Bank,BankState,NAICS,Term,NoEmp,NewExist,...,UrbanRural_sc,NoEmpByState,ApprvLoanDiff,LoanGrossPercentage,LogGrAppv,LogSBAAppv,GrossLogApprvRatio,MISMeanByCity,MISMeanByState,LoanApprvByState
0,265151,BATON ROUGE,LA,70802,PLAQUEMINE BK & TR CO,LA,0.0,84.0,12.0,1.0,...,0.0,8.500000,10000.0,90.000000,5.000000,4.954243,0.990849,0.779661,0.819989,40824.127371
1,581236,DALLAS,TX,75214,"BUSINESS LOAN CENTER, LLC",SC,0.0,240.0,4.0,1.0,...,0.0,11.772419,125000.0,76.747856,5.698970,5.574031,0.978077,0.793694,0.809681,114567.073176
2,420853,ROCHESTER,NY,14617,FIRST NIAGARA BANK NATL ASSOC,NY,811310.0,60.0,1.0,1.0,...,0.5,7.549685,7500.0,50.000000,4.176091,3.875061,0.927916,0.854041,0.801759,39007.628449
3,770643,ROCHESTER,NY,14625,CITIZENS BANK NATL ASSOC,RI,423390.0,60.0,1.0,1.0,...,0.5,5.521338,20000.0,50.000000,4.602060,4.301030,0.934588,0.854041,0.801759,25312.442725
4,268102,ATTICA,NY,14011,FIVE STAR BANK,NY,444130.0,189.0,13.0,2.0,...,0.0,21.813333,82062.0,83.750099,5.703291,5.626277,0.986496,0.937500,0.801759,39007.628449


## Processing Test data

In [197]:
def test_processing(df):
    encode_below_10(df)
    encode_above_10(df)
    df_feature_engineering(df)
    
    return df
    

In [198]:
test_df = test_processing(test_df)

In [199]:
test_df.head()

,index,City,State,Zip,Bank,BankState,NAICS,Term,NoEmp,NewExist,...,BankState_trg,NoEmpByState,ApprvLoanDiff,LoanGrossPercentage,LogGrAppv,LogSBAAppv,GrossLogApprvRatio,MISMeanByCity,MISMeanByState,LoanApprvByState
26743,754920,GOSHEN,KY,40026,"PNC BANK, NATIONAL ASSOCIATION",DE,722211,120,30,1.0,...,0.747890,9.104886,76275.0,75.0,5.484442,5.359503,0.977219,0.777778,0.783505,45944.081598
246718,479325,CITRUS HEIGHT,CA,95621,WESTAMERICA BANK,CA,235110,300,13,1.0,...,0.781248,10.663594,28000.0,80.0,5.146128,5.049218,0.981168,1.000000,0.814619,56019.355384
354854,800789,AUSTIN,TX,78759,BANK OF AMERICA NATL ASSOC,NC,423690,84,1,2.0,...,0.706960,6.217258,5000.0,50.0,4.000000,3.698970,0.924743,0.855186,0.817320,28007.349316
510350,705782,ADDISON,IL,60101,BMO HARRIS CENT. NATL ASSOC,IL,422690,60,2,1.0,...,0.777357,5.571429,10000.0,80.0,4.698970,4.602060,0.979376,0.897959,0.774519,34670.626263
126775,538511,ANDERSON,IN,46016,STAR FINANCIAL BANK,IN,323110,60,2,1.0,...,0.904049,7.129630,13800.0,80.0,4.838849,4.741939,0.979973,0.888889,0.835208,39916.442782


In [ ]:
# Scaling the test dataset
